In [ ]:
from evaluation_metric import *
from baseline_model import *
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_data = pd.read_pickle('Data/train_data.pkl')
train_labels = pd.read_pickle('Data/train_labels.pkl').loc[train_data.index]
test_data = pd.read_pickle('Data/test_data.pkl')
Features = train_data.columns
train_data.shape, train_labels.shape, test_data.shape

In [ ]:
categorical_features = ['B_30', 'B_31', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
train_data.drop(columns=['S_2'], axis=1, inplace=True)
test_data.drop(columns=['S_2'], axis=1, inplace=True)
train_data[categorical_features] = train_data[categorical_features].astype(str)
test_data[categorical_features] = test_data[categorical_features].astype(str)
numerical_features = list(set(train_data.columns).difference(set(categorical_features)))

In [ ]:
enc = OneHotEncoder()
train_encoded_df = enc.fit_transform(train_data[categorical_features])
test_encoded_df = enc.transform(test_data[categorical_features])

In [ ]:
column_name = []
for i in range(0, train_encoded_df.shape[1], 1):
    column_name.append('enc_' + str(i))
train_enc_df = pd.DataFrame(train_encoded_df.toarray(), columns = column_name, index= train_data.index)

column_name = []
for i in range(0, test_encoded_df.shape[1], 1):
    column_name.append('enc_' + str(i))
test_enc_df = pd.DataFrame(test_encoded_df.toarray(), columns = column_name, index= test_data.index)

In [ ]:
train_data.drop(columns=categorical_features, axis=1, inplace=True)
test_data.drop(columns=categorical_features, axis=1, inplace=True)

In [ ]:
train_test_data = pd.concat([train_data,test_data], axis = 0)

In [ ]:
scaler = StandardScaler()
train_test_scaled = scaler.fit_transform(train_test_data.fillna(train_test_data.median()))

pca = PCA(n_components=30, random_state=42)
train_test_scaled_PCA = pca.fit_transform(train_test_scaled)

print(pca.explained_variance_ratio_)

In [ ]:
column_name = []
for i in range(0, train_test_scaled_PCA.shape[1], 1):
    column_name.append('PCA_' + str(i))

train_test_PCA_df = pd.DataFrame(train_test_scaled_PCA, columns= column_name, index=train_test_data.index)

In [ ]:
train_data['unique_id'] = range(0, train_data.shape[0],1)
train_test_PCA_df['unique_id'] = range(0, train_test_PCA_df.shape[0],1)
train_enc_df['unique_id'] = range(0, train_enc_df.shape[0],1)
test_enc_df['unique_id'] = range(0, test_enc_df.shape[0],1)
test_data['unique_id'] = range(0, test_data.shape[0],1)

In [ ]:
train_data.set_index('unique_id', inplace=True)
train_test_PCA_df.set_index('unique_id', inplace=True)
train_enc_df.set_index('unique_id', inplace=True)
test_enc_df.set_index('unique_id', inplace=True)
test_data.set_index('unique_id', inplace=True)

In [ ]:
train_mo = pd.concat([train_data, train_test_PCA_df.iloc[:5531450+1], train_enc_df], axis=1)
test_mo = pd.concat([test_data, train_test_PCA_df.iloc[5531450+1:].reset_index(), test_enc_df], axis=1)
test_mo.index.name = 'unique_id'
test_mo.drop(columns=['unique_id'], axis=1, inplace=True)

In [ ]:
train_mo.shape, test_mo.shape

In [ ]:
del test_data, train_data, train_test_PCA_df, train_enc_df, test_enc_df

In [ ]:
train_mo.to_pickle('Data/train_mo.pkl')
test_mo.to_pickle('Data/test_mo.pkl')